In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.defchararray import add as npStrAdd
import ipysheet
from ipywidgets import Button, HBox, VBox, Layout, Label

tdata = pd.read_pickle('Transaction.pkl')
bdata = pd.read_pickle('Budget.pkl')
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

rows = 15
columns = 5

In [2]:
def createSheet(tdata, startIndex, rows, columns, key="A"):
    S = ipysheet.sheet(rows = rows, columns = columns, key=key)
    Cells = np.zeros((rows, columns)).tolist()
    for i in range(rows):
        if i + startIndex < tdata.shape[0]:
            Cells[i][0] = ipysheet.cell(i, 0, tdata.iloc[i + startIndex, 0].strftime('%d/%m/%Y'))
            Cells[i][1] = ipysheet.cell(i, 1, tdata.iloc[i + startIndex, 1].astype(str))
            Cells[i][2] = ipysheet.cell(i, 2, tdata.iloc[i + startIndex, 2])
            if tdata.iloc[i + startIndex, 3] == '' or not np.isfinite(tdata.iloc[i + startIndex, 3]):
                Cells[i][3] = ipysheet.cell(i, 3, '')
            else:
                Cells[i][3] = ipysheet.cell(i, 3, tdata.iloc[i + startIndex, 3])

            Cells[i][4] = ipysheet.cell(i, 4, tdata.iloc[i + startIndex, 4])
        else:
            Cells[i][0] = ipysheet.cell(i, 0, '')
            Cells[i][1] = ipysheet.cell(i, 1, '')
            Cells[i][2] = ipysheet.cell(i, 2, '')
            Cells[i][3] = ipysheet.cell(i, 3, '')
            Cells[i][4] = ipysheet.cell(i, 4, '')
            
    return S, Cells
    
def readSheet(tdata, startIndex, rows, columns, S, Cells):
    for i in range(rows):
        if i + startIndex < tdata.shape[0]:
            tdata.iloc[i + startIndex, :] = pd.Series(
                [np.datetime64(pd.to_datetime(Cells[i][0].value, dayfirst=True), 's')] + 
                [float(Cells[i][1].value)] + 
                [Cells[i][2].value, Cells[i][3].value, Cells[i][4].value], 
                index=tdata.columns)
        else:
            if Cells[i][0].value != '':
                row = pd.Series(
                [np.datetime64(pd.to_datetime(Cells[i][0].value, dayfirst=True), 's')] + 
                [float(Cells[i][1].value)] + 
                [Cells[i][2].value, Cells[i][3].value, Cells[i][4].value], 
                index=tdata.columns)

                tdata = tdata.append(row, ignore_index=True)
        
    return tdata

def updateSheet(tdata, startIndex, rows, columns, S, Cells):
    for i in range(rows):
        if i + startIndex < tdata.shape[0]:
            Cells[i][0].value = tdata.iloc[i + startIndex, 0].strftime('%d/%m/%Y')
            Cells[i][1].value = tdata.iloc[i + startIndex, 1].astype(str)
            Cells[i][2].value = tdata.iloc[i + startIndex, 2]
            # print(tdata.iloc[i + startIndex, 3], type(tdata.iloc[i + startIndex, 3]))
            if not isinstance(tdata.iloc[i + startIndex, 3], str) and (tdata.iloc[i + startIndex, 3] == '' or not np.isfinite(tdata.iloc[i + startIndex, 3])):
                Cells[i][3].value = ''
            else:
                Cells[i][3].value = tdata.iloc[i + startIndex, 3]

            Cells[i][4].value = tdata.iloc[i + startIndex, 4]
        else:
            Cells[i][0].value = ''
            Cells[i][1].value = ''
            Cells[i][2].value = ''
            Cells[i][3].value = ''
            Cells[i][4].value = ''
            
    return S, Cells

In [3]:
startIndex = tdata.shape[0] - 5

In [4]:
S, Cells = createSheet(tdata, startIndex, rows, columns, key="A")

In [5]:
next_btn = Button(description='Next', layout=Layout(width='99.5%'))
prev_btn = Button(description='Prev', layout=Layout(width='99.5%'))
save_btn = Button(description='Save', layout=Layout(width='99.5%'))
gmsg = ''
lbl = Label(value='Idle')

def next_click(*args, **kwargs):
    global S
    global Cells
    global tdata
    global startIndex
    global lbl
    lbl.value = 'Working..'
    
    tdata = readSheet(tdata, startIndex, rows, columns, S, Cells)
    
    if(startIndex + 13 > tdata.shape[0]):
        return
    
    startIndex += 13
    S, Cells = updateSheet(tdata, startIndex, rows, columns, S, Cells)
    lbl.value = 'Idle'
    
def prev_click(*args, **kwargs):
    global S
    global Cells
    global tdata
    global startIndex
    global lbl
    lbl.value = 'Working..'
    
    tdata = readSheet(tdata, startIndex, rows, columns, S, Cells)
    
    if(startIndex == 0):
        return
    elif(startIndex - 13 < 0):
        startIndex = 0
    else:
        startIndex -= 13
        
    S, Cells = updateSheet(tdata, startIndex, rows, columns, S, Cells)
    lbl.value = 'Idle'

def save_click(*args, **kwargs):
    global tdata
    global lbl
    lbl.value = 'Working..'
    tdata = readSheet(tdata, startIndex, rows, columns, S, Cells)
    lbl.value = 'Idle'
                      
    # WARNING!! Make certain!
    tdata.to_pickle('Transaction.pkl')
    
prev_btn.on_click(prev_click)
save_btn.on_click(save_click)
next_btn.on_click(next_click)

btns = HBox([prev_btn, save_btn, next_btn])
VBox([S, btns, lbl])

## Raw Functions

In [6]:
# tdata = readSheet(tdata, startIndex, rows, columns, S, Cells)

In [7]:
tdata.iloc[startIndex:, :]

,Date,Amt,Category,TAG,Summary
397,2019-11-08,68.05,Groceries,,Coles
398,2019-11-09,5.70,Eat-Out,,Hungry Jacks
399,2019-11-09,1.10,Eat-Out,,Coles Express
400,2019-11-11,8.50,Eat-Out,,Sushi
401,2019-11-11,12.00,Groceries,,Woolies


In [8]:
# startIndex -= 13
# S, Cells = updateSheet(tdata, startIndex, rows, columns, S, Cells)

In [9]:
# WARNING!! Make certain!
# if False:
#     print('WARNING: Writing Out!!')
#     tdata.to_pickle('Transaction.pkl')

In [10]:
# tdata_bak = tdata.copy()